# Overview

# Preparing Data Client

# Creating Model

In [ ]:
def create_block(x, filters, strides, **kwargs):

    ACTIVATIONS = {
        'relu': layers.ReLU,
        'leaky': layers.LeakyReLU,
        'none': lambda **kwargs : lambda x : x}

    CONV_LAYERS = {
        'conv2d': layers.Conv2D,
        'conv3d': layers.Conv3D,
        'conv2dt': layers.Conv2DTranspose,
        'conv3dt': layers.Conv3DTranspose,
        'none': lambda **kwargs : lambda x : x}

    assert type(filters) is list
    assert type(strides) is list

    # --- Create arg dicts
    kwargs_ = {
        'kernel_size': 3,
        'activations': 'leaky',
        'conv_layers': 'conv3d',
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    kwargs_.update(kwargs)

    for k in kwargs_:
        if type(kwargs_[k]) in [int, str]: 
            kwargs_[k] = [kwargs_[k]] * len(filters)

    kwargs_['filters'] = filters
    kwargs_['strides'] = strides 

    extract = lambda d, index : {k: v[index] for k, v in d.items()}
    kwargs_ = [extract(kwargs_, i) for i in range(len(filters))]

    for k in kwargs_:

        conv_layer = k.pop('conv_layers')
        activation = k.pop('activations')

        assert conv_layer in CONV_LAYERS
        assert activation in ACTIVATIONS

        # --- Create convolutional layer
        x = CONV_LAYERS[conv_layer](**k)(x)

        # --- Create batch norm layer
        x = layers.BatchNormalization()(x)

        # --- Create activation layer
        x = ACTIVATIONS[activation]()(x)

    return x

# Training

# Prediction